<a href="https://colab.research.google.com/github/iis-software/Mitchell_001/blob/master/chatbot_voice_enabled_farsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install --upgrade google-cloud-speech
!export GOOGLE_APPLICATION_CREDENTIALS="avian-principle-125722-c79e6ba2064d.json"
!chmod 644 avian-principle-125722-c79e6ba2064d.json
!pip install TTS
!pip install langdetect
!pip install openai
!pip install wave
!pip install google-cloud-texttospeech



chmod: cannot access 'avian-principle-125722-c79e6ba2064d.json': No such file or directory


In [21]:
!apt-get install -y libasound-dev portaudio19-dev libportaudio2 libportaudiocpp0
!pip install --upgrade pyaudio
!pip install sounddevice


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libasound2-dev' instead of 'libasound-dev'
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
libportaudio2 is already the newest version (19.6.0-1.1).
libportaudiocpp0 is already the newest version (19.6.0-1.1).
portaudio19-dev is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [24]:
import numpy as np
from IPython.display import Audio

# Generate a sine wave
frequency = 440
duration = 3
t = np.linspace(0, duration, int(44100 * duration), endpoint=False)
samples = 0.5 * np.sin(2 * np.pi * frequency * t)

# Display audio in Colab using HTML5 audio tag
Audio(samples, rate=44100)


In [26]:

import pyaudio

p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)

In [27]:
from TTS.config import load_config
from TTS.utils.manage import ModelManager
from TTS.utils.synthesizer import Synthesizer
import openai
from google.cloud import speech
from google.cloud import texttospeech
import os
import pyaudio
import wave
from apikey import apikey
import subprocess
import IPython

from langdetect import detect  # Import language detection library


# Setup OpenAI
openai.api_key = apikey

In [28]:
def record_audio(file_path, record_seconds=5):
    # Record audio from the microphone
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 1
    rate = 44100  # Record at 44100 samples per second
    p = pyaudio.PyAudio()  # Create an interface to PortAudio

    print('Recording...')
    stream = p.open(format=sample_format, channels=channels, rate=rate, frames_per_buffer=chunk, input=True)
    frames = []

    for _ in range(0, int(rate / chunk * record_seconds)):
        data = stream.read(chunk)
        frames.append(data)

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save the recorded data as a WAV file
    with wave.open(file_path, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(sample_format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))

    print('Recording finished.')

In [29]:
def recognize_speech(audio_file_path, language_code='en-US'):  # Set a default language code (e.g., English)
  """
  Recognizes speech from an audio file and returns the text transcript.

  Args:
      audio_file_path (str): Path to the audio file.
      language_code (str, optional): Language code for speech recognition. Defaults to 'en-US' (English).

  Returns:
      str: The recognized text transcript, or an empty string if no speech is recognized.
  """

  client = speech.SpeechClient()  # Replace 'speech' with your library

  with open(audio_file_path, 'rb') as audio_file:
    content = audio_file.read()

  audio = speech.RecognitionAudio(content=content)

  # Set language code in the config object (adjust based on library)
  config = speech.RecognitionConfig(
      language_code=language_code,  # Use the provided language code
      encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16  # Example additional option (adjust based on library)
  )

  try:
    response = client.recognize(config=config, audio=audio)
    if response.results:
      return response.results[0].alternatives[0].transcript
    else:
      print("No speech recognized in audio file.")
      return ""
  except speech.RecognitionError as e:
    print(f"Speech recognition error: {e}")  # Catch specific speech recognition exception

  return ""  # Redundant, can be removed


In [30]:
def chat_with_gpt(prompt):
    response = openai.Completion.create(
        engine="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()

In [31]:
def synthesize_speech(text, output_file, language_code='en-US'):
  """
  This function synthesizes speech from a given text and saves it to an audio file.

  Args:
      text (str): The text to be synthesized.
      output_file (str): Path to the output audio file.
      language_code (str, optional): The language code for speech synthesis. Defaults to 'en-US' (English).
  """
  client = texttospeech.TextToSpeechClient()
  input_text = texttospeech.SynthesisInput(text=text)

  # Set voice parameters based on language code
  voice = texttospeech.VoiceSelectionParams(
      language_code=language_code,
      ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
  )
  audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)

  response = client.synthesize_speech(input=input_text, voice=voice, audio_config=audio_config)
  with open(output_file, 'wb') as out:
    out.write(response.audio_content)


In [32]:
def synthesize_speech_farsi(text, output_file):
    """
    Synthesizes speech for the provided text using the specified TTS tool.

    Args:
        text (str): Text for speech synthesis (without newline characters).
        model_path (str, optional): Path to the TTS model file (if applicable). Defaults to None.
        config_path (str, optional): Path to the TTS configuration file (if applicable). Defaults to None.
        output_path (str, optional): Path to save the generated audio file. Defaults to "speech.wav".
        **kwargs: Additional optional arguments for the TTS tool (e.g., --model_name, --vocoder_name).
    """

    # Ensure text ends without trailing characters (including semicolons and newlines)
    text = text.rstrip(".;\n")  # Remove semicolons (.;), periods (.), and newlines (\n)

    # Check if text length exceeds a limit (replace 100 with actual limit if known)
    if len(text) > 1000:
      print(f"Warning: Text length ({len(text)}) exceeds potential limit (1000).")
      # Consider splitting the text into smaller chunks if necessary

    # Escape single quotes using string comprehension
    escaped_text = ''.join(c for c in text if c != "'" or c.isspace())
    !tts --text "$escaped_text" \
        --model_path "best_model_30824.pth" \
        --config_path "config.json" \
        --out_path $output_file

    #!tts --text "$escaped_text" \
    #    --model_path "best_model_1725.pth" \
    #    --config_path "config_old.json" \
    #    --out_path "speech1.wav"
    return 0



In [33]:
def is_farsi_text(text, threshold=0.7):
  """
  Checks if the provided text is most likely Farsi using language detection with a confidence threshold.

  Args:
      text (str): Text to be checked for language.
      threshold (float, optional): Minimum confidence score for considering Farsi detection valid. Defaults to 0.7.

  Returns:
      bool: True if the detected language is Farsi with sufficient confidence, False otherwise.
  """
  try:
    from langdetect import detect
    detected_language, confidence = detect(text)
    return detected_language == 'fa' and confidence >= threshold  # Check language and confidence
  except langdetect.LangDetectException:  # Catch specific exception
    print("Error during language detection, assuming not Farsi")
    return False
  except:  # Catch other unexpected errors
    print("Unexpected error, assuming not Farsi")
    return False


In [36]:
!pip install wavio

In [44]:
import numpy as np
import os
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio

def recognize_speech(audio_file_path, language_code):
    # Replace with your speech recognition logic
    # This is a placeholder function
    return "Placeholder recognition result"

def chat_with_gpt(user_input):
    # Replace with your ChatGPT interaction logic
    # This is a placeholder function
    return "Placeholder GPT response"

def synthesize_speech(text, output_audio_file):
    # Replace with your speech synthesis logic
    # This is a placeholder function
    print(f"Synthesizing speech for: {text}")

    # Example placeholder for writing to a file (replace with actual synthesis logic)
    audio_data = np.zeros(44100 * 3)  # Placeholder audio data (3 seconds at 44100 Hz)
    write_wav(output_audio_file, 44100, (audio_data * 32767).astype(np.int16))  # Scale to int16 range

if __name__ == "__main__":
    audio_file_path = 'input.wav'  # Adjust filename as per your uploaded file
    output_audio_file = 'speech1.wav'

    # Step 1: User selects language
    while True:
        language = input("Enter language (english/farsi/arabic): ").lower()
        if language in ("english", "farsi", "arabic"):
            break
        else:
            print("Invalid language. Please enter 'english', 'farsi', or 'arabic'.")

    # Step 2: Recognize speech (assuming pre-recorded audio is already uploaded)
    user_input = recognize_speech(audio_file_path, language)
    print("User said:", user_input)

    # Step 3: Get ChatGPT response
    gpt_response = chat_with_gpt(user_input)

    # Step 4: Synthesize speech based on language
    synthesize_speech(gpt_response, output_audio_file)

    # Step 5: Playback synthesized speech
    if os.path.exists(output_audio_file):
        with open(output_audio_file, 'rb') as f:
            audio_data = f.read()

        display(Audio(data=audio_data, rate=44100))  # Adjust rate as needed
    else:
        print(f"Error: File {output_audio_file} not found. Check if the speech synthesis function is saving the file correctly.")


Enter language (english/farsi/arabic): farsi
User said: Placeholder recognition result
Synthesizing speech for: Placeholder GPT response


# New Section

In [ ]:
IPython.display.Audio('speech1.wav')

